# Twitter

In [ ]:
import requests
from requests_oauthlib import OAuth1

In [ ]:
from donthackme import CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET

In [ ]:
# api url
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'

In [ ]:
auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET)

In [ ]:
verify_response = requests.get(url, auth=auth)

In [ ]:
verify_response

In [ ]:
verify_response.headers

In [ ]:
verify_response.json().keys()

In [ ]:
verify_response.json()['name']

In [ ]:
verify_response.json()['screen_name']

# Search

In [ ]:
search_url = 'https://api.twitter.com/1.1/search/tweets.json'

In [ ]:
params = {'q': 'data science',
          'result_type': 'recent'} # popular also possible

In [ ]:
search_response = requests.get(search_url, params=params, auth=auth)

In [ ]:
search_response

In [ ]:
search_response.json()

In [ ]:
re_json = search_response.json()

In [ ]:
first_status = re_json.get('statuses', [{}])[0]
print first_status['text']

In [ ]:
sorted(first_status.keys())

In [ ]:
first_status['user'].keys()

In [ ]:
# okay: re_json['statuses']
re_json.get('statuses', [{}])[0] #advanced

## Streaming API

In [ ]:
from itertools import islice

In [ ]:
r = requests.post('https://stream.twitter.com/1.1/statuses/filter.json',
                 params = {'track': '#data'},
                 auth=auth,
                 stream=True) # important

In [ ]:
tweets = r.iter_lines()

In [ ]:
import json

In [ ]:
for tweet in islice(tweets, 20):
    if tweet != None:
        print json.loads(tweet)['text'][:20]
    else:
        print 'Timeout.'

In [ ]:
r.close()

In [ ]:
# student question: What about the u in u'string'?
print u'Hi, Håvard!'

In [ ]:
u'Hi, Håvard!' # good

In [ ]:
'Hi, Håvard!' # less good!

In [ ]:
type(u'')

In [ ]:
type('')

# Storing things

## Files (e.g. .csv)

- Don't scale (could use HDFS)
- What about JSON?

## Databases

- SQL
- NoSQL

# MongoDB

- Windows: Download 'Community edition'
- MAC
    brew update
    brew install mongodb
    brew services start mongobd
- Linux
    sudo apt install mongodb
    
Plus:

    pip install pymongo

Mongobdb stores BSON.

In [ ]:
from datetime import datetime

In [ ]:
from pymongo import MongoClient

In [ ]:
MongoClient('localhost', 27017)

In [ ]:
MongoClient()

In [ ]:
# stkinf - Database
# music - Collection
c = MongoClient().sb1.music

In [ ]:
c

In [ ]:
jackson = {'name': {'first': 'Michael',
                    'last': 'Jackson',
                    'middle': 'Joseph'},
           'born': datetime(1958, 8, 29),
           'died': datetime(2009, 6, 25),
           'albums': [{'name': 'Thriller',
                       'released': 1982},
                      {'name': 'Bad',
                       'released': 1987}]}

In [ ]:
result = c.insert_one(jackson)

In [ ]:
result

In [ ]:
result.acknowledged

In [ ]:
result.inserted_id

In [ ]:
c.find_one(result.inserted_id)

In [ ]:
c.find_one() # any object

In [ ]:
list(c.find())

In [ ]:
for i in c.find():
    print i

In [ ]:
for i in c.find().limit(10):
    print i

In [ ]:
c.find_one({'born': datetime(1958, 8, 29)})

In [ ]:
c.find_one({'name.middle': 'Joseph'})

In [ ]:
c.find_one({'albums.released': 1982})

In [ ]:
c.find_one({'albums.released': {'$gt': 1980}})

In [ ]:
# AND
c.find_one({'name.first': 'Michael',
             'name.last': 'Bowie'})

In [ ]:
# OR
c.find_one({'$or': [{'name.first': 'Michael'},
                     {'name.last': 'Bowie'}]})

In [ ]:
bowie = {'name': {'first': 'David',
                  'last': 'Bowie',
                  'middle': 'Robert'},
         'born': datetime(1949, 1, 8)}

In [ ]:
c.insert_one(bowie)

In [ ]:
list(c.find({}, {'name.first': 1,
                 'born': 1}))

In [ ]:
list(c.find({}, {'name.first': 1,
                 'born': 1,
                 '_id': 0}))

In [ ]:
list(c.find({}, {'_id': 0}))

In [ ]:
c.find_one({'born': {'$lt': datetime(2000, 1, 1)}, # AND
            'born': {'$gt': datetime(1900, 1, 1)}})

# Modifying things

In [ ]:
result = c.update_one({'name.last': 'Bowie'},
                      {'$set': {'albums': []}})

In [ ]:
result.acknowledged

In [ ]:
c.find_one({'name.last': 'Bowie'})

In [ ]:
c.update_one({'name.last': 'Bowie'},
             {'$push': {'albums': {'name': "Let's Dance",
                                   'released': 1983}}})

In [ ]:
c.find_one({'name.last': 'Bowie'})

In [ ]:
c.update_one({'name.last': 'Bowie'},
             {'$inc': {'albums.0.released': 1}})

In [ ]:
c.find_one({'name.last': 'Bowie'})

In [ ]:
r = c.update_one({'name.last': 'Bowie'},
                 {'$inc': {'albums.0.released': -1}})

## Aggregation

We'll talk more about this next session.

In [ ]:
agg = c.aggregate([{'$group': {'_id': '$born',
                               'people_count': {'$sum': 1}}}])

In [ ]:
list(agg)

# Loading data into Pandas

In [ ]:
import pandas as pd

In [ ]:
pd.io.json.json_normalize(list(c.find({}, {'_id': 0})))

# Web Scraping

## Before we start

- Web scraping can lead to messy data.
- Read websites' terms and conditions.
- Respect websites' robots.txt.
- Be nice!
- The firefox inspector is you friend!

# HTML 101

Wepbages are written in HTML. You can read more on the [w3 html tutorial](http://www.w3schools.com/html/).

We will work with a simple [example page](example.html).

In [ ]:
html = open("example.html").read()

In [ ]:
print html

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(html, "lxml")

In [ ]:
soup.body

In [ ]:
soup.body.ul("li")

In [ ]:
soup("li")

In [ ]:
soup('img')

In [ ]:
soup.img

In [ ]:
for img in soup("img"):
    print img.attrs

In [ ]:
soup.img['src']

In [ ]:
soup.li.parent

In [ ]:
soup.li.parent.name

In [ ]:
soup.li.parent.text

In [ ]:
soup.ul.contents

In [ ]:
soup.ul.children

In [ ]:
list(soup.ul.children)

In [ ]:
soup(lambda tag: tag.has_attr('alt'))

# Scrapy

In [ ]:
import scrapy

In [ ]:
from urlparse import urljoin
class TestSpider(scrapy.Spider):
    name = 'test_spider'
    start_urls = ['http://localhost:8888/files/example.html']
    # can be done from command line
    custom_settings = {'FEED_FORMAT': 'json',
                       'FEED_URI': 'result.json'}
    
    def parse(self, response):
        soup = BeautifulSoup(response.text, 'lxml')
        for list_item in soup('li'):
            yield {'list_item': list_item.text,
                   'url': response.url}
        for link in soup('a'):
            target = urljoin(response.url, link['href'])
            yield scrapy.Request(target, callback=self.parse)

# Running

The spider could now be run using

    scrapy runspider test_spider
   
or

    scrapy runspider --output=results.csv test_spider

Also try

    scrapy shell

In [ ]:
from scrapy.crawler import CrawlerProcess

In [ ]:
process = CrawlerProcess()

In [ ]:
process.crawl(TestSpider)

In [ ]:
process.start()